In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helper import *
import scipy.sparse as sp
from test import load_data_sparse
from SGD import * 
from ALS import *
from Visualization import cv_result
from sklearn.linear_model import LinearRegression
from crossval import *

In [40]:
pred_lst = []

In [3]:
data_name = "data/47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"

ratings, pandas = load_data_sparse(data_name, False)

print(pandas.shape) # (1176952, 3)

stop_criterion = 1e-4
prediction = []
pred_lst = []

(1176952, 3)


In [5]:
# SGD
nb_feature = 40
lambda_ = 0.01
pred_SGD, _, _ = matrix_factorization_SGD_CV(ratings, nb_feature, lambda_, lambda_, stop_criterion)


learn the matrix factorization using SGD...


In [41]:
print(type(pred_SGD))
pred_lst.append(pred_SGD)
print(len(pred_lst))

<class 'numpy.ndarray'>
1


In [ ]:
# ALS
nb_feature = 40
lambda_ = 0.1
pred_ALS, _, _ = ALS_CV(ratings, nb_feature, lambda_, lambda_, stop_criterion)


learn the matrix factorization using ALS...


In [ ]:
pred_lst.append(pred_ALS)
print(len(pred_lst))

In [ ]:
pred_lst_2 = pred_lst.copy()

In [ ]:
len(pred_lst_2)

In [ ]:
simple_models = False

In [ ]:
if simple_models:
    # simple_models
    from simple_models import *
    print("learn global mean")
    pred_globalmean =  global_mean(ratings, 0)
    print(pred_globalmean.shape)
    pred_lst.append(pred_globalmean)

    print("learn user mean")
    pred_usermean = user_mean(ratings, 0)
    print(pred_usermean.shape)
    pred_lst.append(pred_usermean)

    print("learn item mean")
    pred_itemmean = baseline_item_mean(ratings, 0)
    print(pred_itemmean.shape)
    pred_lst.append(pred_itemmean)

In [ ]:
def prepareBlending(ratings, list_pred):
    # determine position of given ratings
    nonzero_row, nonzero_col = ratings.nonzero()
    nonzero_train = list(zip(nonzero_row, nonzero_col))
    
    prediction = []
    prediction_model = []
    
    for pred_id, pred in enumerate(list_pred):
        print(pred_id)
        
        print(pred.shape)
        
        pred_tmp = []
        for i in nonzero_train:
            pred_tmp.append(pred.item(i))

        print(len(pred_tmp)) # (1176952)
        #print(type(pred_tmp))
        
        prediction_model.append(pred_tmp)
    
        if pred_id == 0:
            #prediction_model = pred_tmp
            prediction = pred_tmp
        else:
            prediction = np.vstack((prediction,pred_tmp))
            
        
        print(len(prediction))
        print(type(prediction))
        
    return prediction_model, prediction

In [ ]:
prediction_m, prediction = prepareBlending(ratings, pred_lst)
print(prediction.shape)
print(len(prediction_m))

In [35]:
#pandas.head()
y = pandas['Rating']
y.head()

0    4
1    3
2    4
3    3
4    5
Name: Rating, dtype: int64

In [36]:
## Blending function
X = np.transpose(prediction)
#clf = LinearRegression()
#clf = clf.fit(X, y)
#print('Weights of the different models:', clf.coef_)
#clf.score(X, y)

In [37]:
X[0:5]

array([[3.80625846, 4.01566581, 3.8572805 , 3.6460717 , 4.04347826],
       [3.91891856, 4.03649244, 3.8572805 , 3.88483374, 4.04347826],
       [3.40215908, 3.56430465, 3.8572805 , 3.50505051, 4.04347826],
       [4.42462316, 3.97617039, 3.8572805 , 4.17531798, 4.04347826],
       [4.1365401 , 3.54129924, 3.8572805 , 3.97449909, 4.04347826]])

In [38]:
from sklearn.linear_model import Ridge
linreg = Ridge(alpha=0.1, fit_intercept=False)
linreg.fit(X, y)
linreg.coef_

array([ 0.00420797, -0.01816525,  0.98769168,  0.01037689,  0.01541129])

In [20]:
linreg.score(X, y)

-0.22188844039749542

In [21]:
# compute MSE - Blending
sum((np.round(X.dot(linreg.coef_)) - y)**2)/y.shape[0]

1.5979589651914436

In [22]:
# compute MSE - SGD
sum(np.round(prediction_m[0] - y)**2)/y.shape[0]

1.6271810575112664

In [23]:
# compute MSE - ALS
sum(np.round(prediction_m[1] - y)**2)/y.shape[0]

1.6172902548277246

In [24]:
print(X[0:10])

[[3.80625846 4.01566581]
 [3.91891856 4.03649244]
 [3.40215908 3.56430465]
 [4.42462316 3.97617039]
 [4.1365401  3.54129924]
 [4.49552297 4.49756498]
 [4.08698875 4.06725252]
 [4.35256245 4.02152454]
 [3.66249213 3.5141004 ]
 [3.81558051 3.25561355]]


In [25]:
print(y[0:10])

0    4
1    3
2    4
3    3
4    5
5    4
6    3
7    3
8    2
9    5
Name: Rating, dtype: int64


In [26]:
# final result
result = X.dot(linreg.coef_)
print(result[0:10])

[3.90589547 3.96783546 3.477515   4.16166453 3.79435545 4.47891646
 4.06007006 4.15416886 3.56681228 3.49397265]


In [27]:
print(result.shape)

(1176952,)


In [28]:
if simple_models:
    result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1] + pred_globalmean * linreg.coef_[2] + pred_usermean * linreg.coef_[3] + pred_itemmean * linreg.coef_[4]
else:
    result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1]
        
    

In [23]:
#result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1]

In [44]:
#result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1] + pred_globalmean * linreg.coef_[2] + pred_usermean * linreg.coef_[3] + pred_itemmean * linreg.coef_[4]

In [29]:
result.shape

(10000, 1000)

In [30]:
if simple_models:
    create_submission_from_prediction(result, "out5-0.1.csv")
else:
    create_submission_from_prediction(result, "out2-0.1.csv")

In [39]:
create_submission_from_prediction(pred_ALS, "out-onlyALS.csv")